In [2]:
import pyodbc
from piLang.piLang.SQLTools import SQLTools
from prettytable import PrettyTable
from piLang.piLang.BoxPlot import BoxPlot
from piLang.piLang.LangValidator import LangValidator, PatternFormat
from validator.BabyCodeValidator import BabyCodeValidator
import time

"""
This code will extract and validate the Baby code record in accordance with the QH PDC file format spec.

"""
cnxn = pyodbc.connect('Driver={ODBC Driver 13 for SQL Server}; server=localhost\SQLEXPRESS; database=MaterDB; uid=Shane; password=; trusted_connection=Yes')
cursor = cnxn.cursor()
cursor.execute('SELECT  * FROM dbo.[Babys Birth Code Record] ORDER BY [Mothers UR number]') 
t = SQLTools(cursor)
cnxn.close()
cnxn = None


baby_code_meta = { 
    'Transaction Type':    {'Size': 1,   'Type': "char",     'Mandatory': True,  'Enum': ['N','D']},
    'Mothers UR number':   {'Size': 12,  'Type': "string",   'Mandatory': True,  'Composite': ['%1','facility_id','Date of confinement','Baby number']},
    'Date of confinement': {'Size': 10,  'Type': "date",     'Mandatory': True,  'Format': PatternFormat.DATE_YYMMDD.value},
    'Baby number':         {'Size': 1,   'Type': "int",      'Mandatory': True,  'Max': 9},
    'Code Type':           {'Size': 1,   'Type': "char",     'Mandatory': True,  'Enum':['I', 'A', 'S', 'R', 'T', 'L', 'C', 'M', 'P', 'N', 'F', 'D', 'E', 'B', 'G', 'V']},
    'Babys birth code':    {'Size': 120, 'Type': "string",   'Mandatory': False, 'AllowBlank': True},
    'facility_id':         {'Mandatory': True}
}
# validate the field length

print("\n---> Running langvalidator ...")
stime = time.time()
lang_validator = LangValidator(t.rs, baby_code_meta)
lang_validator.validate()
lang_validator.counters.calcMean()
lang_validator.counters.calcPercent()
print("Completed in " + str(time.time() - stime) + " secs")
print(lang_validator.getErrors())
print(lang_validator.counters)

print("\n---> Running PDCValidator ...")
stime = time.time()
pdc_validator = BabyCodeValidator(t.rs, baby_code_meta)
pdc_validator.validate()
pdc_validator.counters.calcMean()
pdc_validator.counters.calcPercent()
print("Completed in " + str(time.time() - stime) + " secs")
print(pdc_validator.getErrors())
print(pdc_validator.counters)

counters = list()
counters.extend(lang_validator.counters.toList())
counters.extend(pdc_validator.counters.toList())

#print("\n--> Plot:")

x = BoxPlot()
x.plot(counters)

print("\n\n### DONE ###")



---> Running langvalidator ...
Completed in 20.782410383224487 secs
Error: Duplicate composite key: 'Mothers UR number+facility_id+Date of confinement+Baby number', value: '6297266.0318.028/01/20191'
Error: Duplicate composite key: 'Mothers UR number+facility_id+Date of confinement+Baby number', value: '6297266.0318.028/01/20191'
Error: Duplicate composite key: 'Mothers UR number+facility_id+Date of confinement+Baby number', value: '6297266.0318.028/01/20191'
Error: Duplicate composite key: 'Mothers UR number+facility_id+Date of confinement+Baby number', value: '6297266.0318.028/01/20191'
Error: Duplicate composite key: 'Mothers UR number+facility_id+Date of confinement+Baby number', value: '6297266.0318.028/01/20191'
Error: Duplicate composite key: 'Mothers UR number+facility_id+Date of confinement+Baby number', value: '6297266.0318.028/01/20191'
Error: Duplicate composite key: 'Mothers UR number+facility_id+Date of confinement+Baby number', value: '6297405.0318.028/01/20191'
Error: 



### DONE ###
